In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## data processing

### unzip raw mp3

In [ ]:
%cd /content/drive/MyDrive/voicevishing_audio
!unzip -qq "/content/drive/MyDrive/voicevishing_audio/voicevishingmp3file.zip"

/content/drive/MyDrive/voicevishing_audio/raw_audio


### installing pydub

In [ ]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pydub import AudioSegment
import os

### removing corrupted or bad data

In [ ]:
directory = "/content/drive/MyDrive/voicevishing_audio/mp3file/"

for filename in os.listdir(directory):
    if filename.endswith('.mp3'):
        try:
            AudioSegment.from_file(os.path.join(directory, filename), format="mp3")
        except Exception as e:
            print(f"Error reading {filename}: {e}")
            os.remove(os.path.join(directory, filename))
            print(f"{filename} removed.")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[mp3float @ 0x55ce0b32bf00] Header missing
Error while decoding stream #0:0: Invalid data found when processing input
[mp3float @ 0x55ce0b32bf00] Header missing
Error while decoding stream #0:0: Invalid data found when processing input
[mp3float @ 0x55ce0b32bf00] Header missing
Error while decoding stream #0:0: Invalid data found when processing input
[mp3float @ 0x55ce0b32bf00] Header missing
Error while decoding stream #0:0: Invalid data found when processing input
[mp3float @ 0x55ce0b32bf00] Header missing
Error while decoding stream #0:0: Invalid data found when processing input
[mp3float @ 0x55ce0b32bf00] Header missing
Error while decoding stream #0:0: Invalid data found when processing input
[mp3float @ 0x55ce0b32bf00] Header missing
Error while decoding stream #0:0: Invalid data found when processing input
[mp3float @ 0x55ce0b32bf00] Header missing
Error while decoding stream #0:0: Invalid data found when processing input
[mp3float @ 0x55ce0b

### increasing dB

In [ ]:
directory = "/content/drive/MyDrive/voicevishing_audio/mp3file/"
export_directory = "/content/drive/MyDrive/voicevishing_audio/mp3file_10dbup/"

for filename in os.listdir(directory):
  sound = AudioSegment.from_file(directory+filename, format="mp3")

  increased_dB = sound + 10
  newname = export_directory+filename[:-4]+"_10dbup.mp3"
  increased_dB.export(newname, format="mp3")
  print(newname+"converted")

/content/drive/MyDrive/voicevishing_audio/mp3file_10dbup/voice_file1_10dbup.mp3converted
/content/drive/MyDrive/voicevishing_audio/mp3file_10dbup/voice_file2_10dbup.mp3converted
/content/drive/MyDrive/voicevishing_audio/mp3file_10dbup/voice_file3_10dbup.mp3converted
/content/drive/MyDrive/voicevishing_audio/mp3file_10dbup/voice_file4_10dbup.mp3converted
/content/drive/MyDrive/voicevishing_audio/mp3file_10dbup/voice_file5_10dbup.mp3converted
/content/drive/MyDrive/voicevishing_audio/mp3file_10dbup/voice_file6_10dbup.mp3converted
/content/drive/MyDrive/voicevishing_audio/mp3file_10dbup/voice_file7_10dbup.mp3converted
/content/drive/MyDrive/voicevishing_audio/mp3file_10dbup/voice_file9_10dbup.mp3converted
/content/drive/MyDrive/voicevishing_audio/mp3file_10dbup/voice_file8_10dbup.mp3converted
/content/drive/MyDrive/voicevishing_audio/mp3file_10dbup/voice_file10_10dbup.mp3converted
/content/drive/MyDrive/voicevishing_audio/mp3file_10dbup/voice_file11_10dbup.mp3converted
/content/drive/MyDr

### convert to wav

In [ ]:
directory = "/content/drive/MyDrive/voicevishing_audio/mp3file_10dbup/"
export_directory = "/content/drive/MyDrive/voicevishing_audio/wavfile/"

i = int(0)
for filename in os.listdir(directory):
  i += 1
  dst = export_directory + "/audio%d.wav"%i
  audSeg = AudioSegment.from_mp3(directory+filename)
  audSeg.export(dst, format="wav")


### convert to text file

In [ ]:
! pip install SpeechRecognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 8.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1


In [ ]:
import os
import speech_recognition as sr
import wave

r = sr.Recognizer()

input_dir = "/content/drive/MyDrive/voicevishing_audio/wavfile"
output_dir = "/content/drive/MyDrive/voicevishing_audio/vishing_text"

segment_length = 30 #sec

wav_files = [f for f in os.listdir(input_dir) if f.endswith('.wav')]

for wav_file in wav_files:
    print("Processing file: ", wav_file)

    # get the duration of the WAV file in seconds
    with wave.open(os.path.join(input_dir, wav_file), 'r') as wave_file:
        audio_duration = wave_file.getnframes() / float(wave_file.getframerate())

    # calculate the number of segments
    num_segments = int(audio_duration / segment_length) + 1

    # loop over each segment
    for i in range(num_segments):
        print("Processing segment: ", i+1)

        start_time = i * segment_length
        end_time = min((i+1) * segment_length, audio_duration)

        # load the audio segment
        with sr.AudioFile(os.path.join(input_dir, wav_file)) as source:
            audio_segment = r.record(source, offset=start_time, duration=end_time-start_time)

        try:
            text = r.recognize_google(audio_segment, language='ko')
            print("Text: ", text)

            text_file = os.path.splitext(wav_file)[0] + f"_segment_{i+1}.txt"
            with open(os.path.join(output_dir, text_file), "w") as f:
                f.write(text)
                print("Text file saved!")
        except sr.UnknownValueError:
            print("Could not understand audio")
        except sr.RequestError as e:
            print("Could not request results; {0}".format(e))


### text processing

5자 이하 삭제<br>
\t해서 뒤에 1붙이기<br>
세로축 병합? 한줄한줄 쓰기<br>
neg_text와 한줄한줄씩 랜덤으로 병합<br>

In [ ]:
dir_path = '/content/drive/MyDrive/voicevishing_audio/vishing_text'
min_length = 5

for filename in os.listdir(dir_path):
  if filename.endswith('.txt'):
    file_path = os.path.join(dir_path, filename)

    with open(file_path, 'r', encoding='utf-8') as file_:
      file_contents = file_.read()
      if len(file_contents) < min_length:
        os.remove(file_path)
      elif not file_contents.endswith('\t1'):
        with open(file_path, 'a', encoding='utf-8') as output_file:
          output_file.write('\t1')

In [ ]:
dir_path = '/content/drive/MyDrive/voicevishing_audio/vishing_text'
output_file = '/content/drive/MyDrive/voicevishing_audio/pos.txt'

with open(output_file, 'w', encoding='utf-8') as out_file:
    for filename in os.listdir(dir_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(dir_path, filename)

            with open(file_path, 'r') as in_file:
                file_contents = in_file.read()
                out_file.write(file_contents.strip() + '\n')

In [ ]:
dir = '/content/drive/MyDrive/voicevishing_audio/'
with open(dir+'pos.txt', 'r', encoding='utf-8') as file:
  pos_cnt = 0
  for line in file:
    pos_cnt += 1
print(f'pos text contains {pos_cnt} line')

with open(dir+'neg_text.txt', 'r', encoding='utf-8') as file:
  neg_cnt = 0
  for line in file:
    neg_cnt += 1
print(f'neg text contains {neg_cnt} line')

pos text contains 5181 line
neg text contains 100000 line


In [ ]:
import random

with open(dir+'neg_text.txt', 'r', encoding='utf-8') as file:
  lines = file.readlines()

num_lines_to_del = abs(neg_cnt-pos_cnt)
print(f'will delte {num_lines_to_del} lines')

lines_to_del = random.sample(range(neg_cnt), num_lines_to_del)

with open(dir+'neg_text_pruned.txt', 'w', encoding='utf-8') as pruned_file:
  for i, line in enumerate(lines):
    if i in lines_to_del:
      continue
    pruned_file.write(line)

will delte 94819 lines


In [ ]:
dir = '/content/drive/MyDrive/voicevishing_audio/'

with open(dir+'pos.txt', 'r', encoding='utf-8') as f1, open(dir+'neg_text_pruned.txt', 'r', encoding='utf-8') as f2:
    lines = f1.readlines() + f2.readlines()

random.shuffle(lines)

with open(dir+'voice_phishing.txt', 'w', encoding='utf-8') as f:
    for i, line in enumerate(lines):
        if i % 2 == 0:
            f.write(line.strip() + '\n')
        else:
            f.write(line.strip() + '\n')


## deep learning model making

### import

In [9]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, Flatten, LSTM, Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import numpy as np

### load dataset and tokenize and split

In [3]:
def load_dataset(file_path):
  texts = []
  labels = []
  with open(file_path, 'r') as f:
    for line in f:
      parts = line.strip().split('\t')
      texts.append(parts[0])
      labels.append(int(parts[1]))
  return texts, labels

In [4]:
dir = '/content/drive/MyDrive/voicevishing_audio/'
texts, labels = load_dataset(dir+'voice_phishing.txt')

labels = np.array(labels)
num_classes = len(set(labels))
labels = np.eye(num_classes)[labels]

### tokenize

In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
max_sequence_length = 50
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

In [6]:
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1
indices = np.arange(len(texts))
np.random.shuffle(indices)
train_indices = indices[:int(train_ratio * len(texts))]
val_indices = indices[int(train_ratio * len(texts)):int((train_ratio+val_ratio) * len(texts))]
test_indices = indices[int((train_ratio+val_ratio) * len(texts)):]

train_data = padded_sequences[train_indices]
train_labels = labels[train_indices]
val_data = padded_sequences[val_indices]
val_labels = labels[val_indices]
test_data = padded_sequences[test_indices]
test_labels = labels[test_indices]

### make model

In [10]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=50, input_length=max_sequence_length))
model.add(Bidirectional(LSTM(units=128, dropout=0.2)))
model.add(Dropout(0.5))
model.add(Dense(units=num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 50, 50)            2043700   
                                                                 
 bidirectional (Bidirectiona  (None, 256)              183296    
 l)                                                              
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 2)                 514       
                                                                 
Total params: 2,227,510
Trainable params: 2,227,510
Non-trainable params: 0
_________________________________________________________________


### train

In [11]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

In [12]:
model.fit(
    train_data,
    train_labels,
    epochs=10,
    batch_size=32,
    validation_data=(val_data, val_labels),
    callbacks=[early_stopping]
)

Epoch 1/10
260/260 [==============================] - 49s 149ms/step - loss: 0.2646 - accuracy: 0.8918 - val_loss: 0.1054 - val_accuracy: 0.9604
Epoch 2/10
260/260 [==============================] - 12s 46ms/step - loss: 0.0537 - accuracy: 0.9821 - val_loss: 0.0741 - val_accuracy: 0.9720
Epoch 3/10
260/260 [==============================] - 8s 32ms/step - loss: 0.0132 - accuracy: 0.9965 - val_loss: 0.1101 - val_accuracy: 0.9662
Epoch 4/10
260/260 [==============================] - 4s 17ms/step - loss: 0.0157 - accuracy: 0.9952 - val_loss: 0.0878 - val_accuracy: 0.9662
Epoch 5/10
260/260 [==============================] - 5s 21ms/step - loss: 0.0032 - accuracy: 0.9996 - val_loss: 0.1067 - val_accuracy: 0.9681


### evaluate

In [13]:
test_loss, test_acc = model.evaluate(test_data, test_labels, batch_size=32)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

33/33 [==============================] - 0s 6ms/step - loss: 0.0900 - accuracy: 0.9769
Test loss: 0.08997146785259247
Test accuracy: 0.9768562912940979


### save

In [14]:
dir = '/content/drive/MyDrive/voicevishing_audio/'
model.save(dir+'voice_phishing_model.h5')

### TEST

In [15]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 KB 38.4 MB/s eta 0:00:00


In [16]:
from konlpy.tag import Okt
from string import punctuation
import re
from keras.models import load_model
import numpy as np

In [17]:
okt = Okt()

def preprocess_text(text):
    okt = Okt()
    morphemes = okt.morphs(text)
    return ' '.join(morphemes)

In [18]:
model = load_model(dir+'voice_phishing_model.h5')

def predict_phishing(text):
    preprocessed_text = preprocess_text(text)
    sequence = tokenizer.texts_to_sequences([preprocessed_text])
    padded_sequence = pad_sequences(sequence, maxlen=50)
    predictions = model.predict(padded_sequence)
    phishing_probability = predictions[0][1]
    is_phishing = int(phishing_probability > 0.5)
    return is_phishing, phishing_probability

In [19]:
text=input()
predict, prob = predict_phishing(text)
if predict > 0.5:
  print(f"{prob} 확률로 보이스피싱")
else:
  print(f"{1-prob} 확률로 보이스피싱 아님")

서울중앙지방 검찰청에서 전화드린 거에요 현재 저희 검찰에 금융범죄사기단을 검거한 상태에 이 검거과정에서 대량의 신용카드와 대포통장을 입수하던 과정에
1/1 [==============================] - 2s 2s/step
0.9998936653137207 확률로 보이스피싱
